In [81]:
import torch

In [82]:
tensor = torch.tensor([0.1, -0.2, -0.3])

In [83]:
X = tensor.reshape(1,3)

In [84]:
X

tensor([[ 0.1000, -0.2000, -0.3000]])

In [85]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(in_features=3, out_features=1, bias=True)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        return x

In [119]:
model = Net()

In [151]:
par = []
for parameter in model.parameters():
    print(id(parameter))
    par.append(parameter)   

139969208633528
139969208635256


In [152]:
id(par[1])

139969208635256

In [153]:
par

[Parameter containing:
 tensor([[0.4493, 0.3461, 0.5556]], requires_grad=True), Parameter containing:
 tensor([0.4637], requires_grad=True)]

In [154]:
weight = torch.tensor([0.4493, 0.3461, 0.5556])
weight = weight.reshape(1,3)

In [155]:
bias = torch.tensor([0.46373])

In [156]:
par[0].data = torch.nn.Parameter(weight)
par[1].data = torch.nn.Parameter(bias)

In [157]:
id(par[0])

139969208633528

In [158]:
par[1].shape

torch.Size([1])

In [159]:
X.matmul(par[0].t()) + par[1]

tensor([[0.2728]], grad_fn=<AddBackward0>)

In [160]:
torch.nn.functional.relu(X.matmul(par[0].t()) + par[1])

tensor([[0.2728]], grad_fn=<ReluBackward0>)

In [161]:
model(X)

tensor([[0.2728]], grad_fn=<ReluBackward0>)

In [164]:
torch.nn.functional.relu(torch.tensor([-0.1, 0.2]))

tensor([0.0000, 0.2000])

In [165]:
import torch
from torch.optim import Optimizer
class _RequiredParameter(object):
    """Singleton class representing a required parameter for an Optimizer."""
    def __repr__(self):
        return "<required parameter>"

required = _RequiredParameter()




class SGD(Optimizer):
    r"""Implements stochastic gradient descent (optionally with momentum).
    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf
    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.
        Considering the specific case of Momentum, the update can be written as
        .. math::
                  v = \rho * v + g \\
                  p = p - lr * v
        where p, g, v and :math:`\rho` denote the parameters, gradient,
        velocity, and momentum respectively.
        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form
        .. math::
             v = \rho * v + lr * g \\
             p = p - v
        The Nesterov version is analogously modified.
    """

    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                print('Parameter before update:',p)
                print('Gradient:',d_p)
                print('X:',X)
                print('Learning rate:',-group['lr'])
                check = p.clone()
                p.data.add_(-group['lr'], d_p)
                print('Parameter of the tensor after update:', p)
                print('Sanity check:', check +(-group['lr']) * d_p)

        return loss

In [166]:
X.shape

torch.Size([1, 3])

In [167]:
par[1].grad

In [168]:
optimizer_detail = SGD(model.parameters(), lr=1)

In [169]:
criterion = torch.nn.MSELoss()


In [170]:
output = model(X)

In [171]:
loss = criterion(1, output)

In [172]:
optimizer_detail.zero_grad()

In [173]:
loss.backward()

In [174]:
optimizer_detail.step()

Parameter before update: Parameter containing:
tensor([[0.4493, 0.3461, 0.5556]], requires_grad=True)
Gradient: tensor([[-0.1454,  0.2909,  0.4363]])
X: tensor([[ 0.1000, -0.2000, -0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[0.5947, 0.0552, 0.1193]], requires_grad=True)
Sanity check: tensor([[0.5947, 0.0552, 0.1193]], grad_fn=<AddBackward0>)
Parameter before update: Parameter containing:
tensor([0.4637], requires_grad=True)
Gradient: tensor([-1.4545])
X: tensor([[ 0.1000, -0.2000, -0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([1.9182], requires_grad=True)
Sanity check: tensor([1.9182], grad_fn=<AddBackward0>)


In [175]:
loss

tensor(0.5289, grad_fn=<MeanBackward1>)

In [176]:
model(X)

tensor([[1.9309]], grad_fn=<ReluBackward0>)

In [177]:
output

tensor([[0.2728]], grad_fn=<ReluBackward0>)

In [178]:
(output-1)**2

tensor([[0.5289]], grad_fn=<PowBackward0>)

# Calculations
f(output-1) = (output-1) ** 2 = 0.5289

f'(output-1) = 2 * (output-1) = -1.4545

d/dw((relu(wx+b)-1)**2) = (-1.4545) * 1 * [ 0.1000, -0.2000, -0.3000] = [-0.1454,  0.2909,  0.4363], which is perfecty played with statement above:

Gradient: tensor([[-0.1454,  0.2909,  0.4363]])


In [179]:
(output-1)*2

tensor([[-1.4545]], grad_fn=<MulBackward0>)

In [32]:
model(X)

tensor([[1.9885]], grad_fn=<AddmmBackward>)